# Using internal_utils functions from the `jmspack` package to manipulate the postgresql back end of `jms_tracker`

## Possible internal_utils functions

- postgresql_data_extraction()
- postgresql_table_names_list()
- create_postgresql_table_based_on_df()
- add_data_to_postgresql_table()
- delete_postgresql_table()

In [64]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from jmspack.internal_utils import (postgresql_data_extraction,
                                    postgresql_table_names_list,
                                   create_postgresql_table_based_on_df,
                                    add_data_to_postgresql_table,
                                   delete_postgresql_table)
from datetime import datetime

In [4]:
load_dotenv(".env")

True

## Extract table names from an external postgresql database

In [5]:
table_list = postgresql_table_names_list(database_name = 'tracker',
                                         user='tracker',)
table_list

[('django_session',),
 ('iris_test',),
 ('suggested_energy_intake',),
 ('sleep_data',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]

## Create an external postgresql table based on a pandas dataframe

PUT response = {
barcode: int64,
amount: float32,
}

In [15]:
amount_df = pd.DataFrame({"user_id": 1,
    "timestamp": datetime.now(),
    "barcode": 8714779001053, 
             "amount": 1.2}, index=[0])

In [16]:
create_postgresql_table_based_on_df(df=amount_df,
                         database_name="tracker",
                             user="tracker",
                            table_name="foodIntakes",
                            )

'CREATE TABLE foodIntakes (\n                user_id int,\ntimestamp timestamp,\nbarcode int,\namount float\n    )'

GET foodCodes
response = [{
name: str,
barcode: int64,
}]

In [71]:
codes_df = pd.DataFrame({"name": "Falafel",
    "barcode": 8714779001053}, index=[0])

In [72]:
create_postgresql_table_based_on_df(df=codes_df,
                         database_name="tracker",
                             user="tracker",
                            table_name="foodCodes",
                            )

'CREATE TABLE foodCodes (\n                name text,\nbarcode int\n    )'

## Add data from a pandas dataframe to an external postgresql table

In [35]:
amount_df.astype({"user_id": int,
                 "barcode": int}).dtypes

user_id               int64
timestamp    datetime64[ns]
barcode               int64
amount              float64
dtype: object

In [62]:
def temp_add_data_to_postgresql_table(
    df: pd.DataFrame,
    database_name: str,
    user: str,
    table_name: str,
):
    columns_string = ", ".join(df.columns.tolist())
    value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])
    insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})"""
    try:
        conn = psycopg2.connect(
            host=os.getenv("postgresql_host"),
            database=database_name,
            user=user,
            password=os.getenv("postgresql_password"),
        )
        cursor = conn.cursor()
        for oid in range(0, df.shape[0]):
            value_tuple = tuple(str(df.loc[oid, df.columns[col_number]]) 
                          if "numpy" in str(type(df.loc[oid, df.columns[col_number]])) 
                          else df.loc[oid, df.columns[col_number]] 
                          for col_number in range(0, df.shape[1]))
            _ = cursor.execute(insert_string, value_tuple)

        _ = conn.commit()
        _ = conn.close()

    except psycopg2.Error as e:
        print(e)

    return insert_string

In [76]:
database_name="tracker"
user="tracker"
table_name="foodcodes"

In [77]:
df=codes_df

In [89]:
columns_string = ", ".join(df.columns.tolist())
value_placeholder_string = ", ".join(["%s" for x in range(0, df.shape[1])])
insert_string = f"""INSERT INTO {table_name} ({columns_string}) VALUES ({value_placeholder_string})"""
insert_string

'INSERT INTO foodcodes (name, barcode) VALUES (%s, %s)'

In [90]:
try:
    conn = psycopg2.connect(
        host=os.getenv("postgresql_host"),
        database=database_name,
        user=user,
        password=os.getenv("postgresql_password"),
    )
    cursor = conn.cursor()
    value_tuple = ("Hummus pikant", str(8711852200181))
    _ = cursor.execute(insert_string, value_tuple)

    _ = conn.commit()
    _ = conn.close()

except psycopg2.Error as e:
    print(e)

In [ ]:
amount_df.user_id[0]

In [83]:
temp_add_data_to_postgresql_table(df=codes_df,
                         database_name="tracker",
                             user="tracker",
                            table_name="foodcodes",
                            )

can't adapt type 'numpy.int64'


'INSERT INTO foodcodes (name, barcode) VALUES (%s, %s)'

## Read in data from an external postgresql database

In [91]:
df = postgresql_data_extraction(table_name = 'foodcodes',
                                database_name = 'tracker',
                                user='tracker',)

In [97]:
df.to_json(orient="records")

'[{"name":"Falafel","barcode":"8714779001053                   "},{"name":"Hummus pikant","barcode":"8711852200181                   "}]'

In [11]:
# df = postgresql_data_extraction(table_name = 'suggested_energy_intake',
#                                 database_name = 'tracker',
#                                 user='tracker',)

df = postgresql_data_extraction(table_name = 'iris_test',
                                database_name = 'tracker',
                                user='tracker',)

In [12]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


## Delete an external postgresql database

#### Printing table names before and after to check removal occurs

In [10]:
postgresql_table_names_list(database_name = 'tracker',
                                         user='tracker',)

[('django_session',),
 ('iris_test',),
 ('suggested_energy_intake',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]

In [11]:
delete_postgresql_table(database_name="tracker",
                             user="tracker",
                            table_name="iris_test",)

'iris_test has been deleted from tracker'

In [12]:
postgresql_table_names_list(database_name = 'tracker',
                                         user='tracker',)

[('django_session',),
 ('suggested_energy_intake',),
 ('django_migrations',),
 ('django_content_type',),
 ('auth_permission',),
 ('auth_group',),
 ('auth_group_permissions',),
 ('auth_user_groups',),
 ('auth_user_user_permissions',),
 ('auth_user',),
 ('django_admin_log',)]